## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,75.36,94,20,5.75,few clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,84.16,82,32,11.01,scattered clouds
2,2,Georgetown,MY,5.4112,100.3354,82.33,98,20,3.44,few clouds
3,3,San Quintin,MX,30.4833,-115.9500,66.20,64,5,13.80,clear sky
4,4,Pacific Grove,US,36.6177,-121.9166,60.91,81,100,8.01,overcast clouds


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?0
What is the maximum temperature you would like for your trip?100


In [24]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                        (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,75.36,94,20,5.75,few clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,84.16,82,32,11.01,scattered clouds
2,2,Georgetown,MY,5.4112,100.3354,82.33,98,20,3.44,few clouds
3,3,San Quintin,MX,30.4833,-115.9500,66.20,64,5,13.80,clear sky
4,4,Pacific Grove,US,36.6177,-121.9166,60.91,81,100,8.01,overcast clouds


In [25]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [26]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,75.36,94,20,5.75,few clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,84.16,82,32,11.01,scattered clouds
2,2,Georgetown,MY,5.4112,100.3354,82.33,98,20,3.44,few clouds
3,3,San Quintin,MX,30.4833,-115.9500,66.20,64,5,13.80,clear sky
4,4,Pacific Grove,US,36.6177,-121.9166,60.91,81,100,8.01,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
685,685,La Ronge,CA,55.1001,-105.2842,53.82,62,40,13.80,scattered clouds
686,686,Menongue,AO,-14.6585,17.6910,58.82,48,0,5.19,clear sky
687,687,Kattivakkam,IN,13.2167,80.3167,87.89,83,48,15.23,scattered clouds
688,688,Mananjary,MG,-21.2167,48.3333,70.18,87,56,8.16,light rain


In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,
1,Puerto Ayora,EC,84.16,scattered clouds,-0.7393,-90.3518,
2,Georgetown,MY,82.33,few clouds,5.4112,100.3354,
3,San Quintin,MX,66.20,clear sky,30.4833,-115.9500,
4,Pacific Grove,US,60.91,overcast clouds,36.6177,-121.9166,
5,Alofi,NU,71.49,overcast clouds,-19.0595,-169.9187,
6,Kodiak,US,54.91,clear sky,57.7900,-152.4072,
7,Cape Town,ZA,60.91,clear sky,-33.9258,18.4232,
8,Olafsvik,IS,47.07,clear sky,64.8945,-23.7142,
9,Ushuaia,AR,37.06,light rain,-54.8000,-68.3000,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:  
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")    
        

In [29]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
689,Wuhan,CN,69.89,overcast clouds,30.5833,114.2667,Novotel Wuhan Xinhua Hotel


In [33]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd> F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))